In [15]:
import numpy as np
from spacer3d.Ripley import load_OIB, load_mask, load_tumor_locations, run_ripley, monte_carlo

In [16]:
### Loading Confocal Image (.OIB)

path = "/data/dkermany_data/Bone_Project/weijie_selected/main_folder/FV10__oibs/FV10__20190620_111343.oib"
# path = "D:\\Datasets\\Bone_Mets\\FV10__oibs\\FV10__20190620_111343\\FV10__20190620_111343.oib"

filename, viewer, steps = load_OIB(path)


In [37]:
from glob import glob
path = "/data/dkermany_data/Bone_Project/weijie_selected/main_folder/FV10__oibs/*.oib"
for f in glob(path):
    load_OIB(f)

KeyboardInterrupt: 

In [17]:
### Loading 3D Binary Mask (.SEG.NRRD)

mask_path = f"/data/dkermany_data/Bone_Project/masks/{filename}.seg.nrrd"
# mask_path = f"D:\\Datasets\\Bone_Mets\\masks\\{filename}.seg.nrrd"

mask = load_mask(mask_path)


(228, 2404, 3376)
uint8
Axes: ZYX


### Loading real data pointsets for comparison

In [18]:
### Loading Tumor Cell Locations (.CSV)
tumor_csv_path = "/data/dkermany_data/Bone_Project/tumor_locations_02_08_2023.csv"
# tumor_csv_path = "D:\\Datasets\\Bone_Mets\\tumor_locations\\tumor_locations_02_08_2023.csv"

tumor_points = load_tumor_locations(tumor_csv_path, filename, steps)

In [ ]:
branch_points_path = f"/data/dkermany_data/Bone_Project/branch_points/{filename}_branch_points.npy"
ng2_points_path = f"/data/dkermany_data/Bone_Project/NG2_Centroids/{filename}_NG2_centroids.npy"
tvc_points_path = f"/data/dkermany_data/Bone_Project/tortuous_segment_centroids/{filename}_tortuous_segment_centroid.npy"
_

In [ ]:
 
ng2_points = np.flip(np.load(ng2_points_path).T, axis=1) # put points into Z,Y,X format (N, 3)
branch_points = np.load(branch_points_path).T # branch points do not need to be flipped to be formatted correctly
tvc_points = np.rint(np.flip(np.load(tvc_points_path).T, axis=1)).astype(int) # put points into Z,Y,X format (N, 3)

In [19]:
print(tumor_points.shape)
print(tumor_points)

(35, 3)
[[  75. 1417. 2404.]
 [  75. 1413. 2395.]
 [  75. 1380. 2377.]
 [  75. 1352. 2479.]
 [  75. 1080. 2266.]
 [  75.  989.  730.]
 [  75.  704. 2427.]
 [  75.  693. 2432.]
 [  75.  732. 2459.]
 [  75.  840. 2633.]
 [  75.  600. 2675.]
 [  75.  595. 2771.]
 [  66. 1432. 2315.]
 [  64.  710. 2384.]
 [  62.  721. 2351.]
 [  61.  747. 2518.]
 [  59. 1119. 2344.]
 [  58. 1399. 2301.]
 [  53.  647. 1892.]
 [  53.  647. 1892.]
 [  51. 1417. 2336.]
 [  51. 1423. 2351.]
 [  48.  592. 2799.]
 [  43. 1393. 2367.]
 [  42.  685. 1942.]
 [  36. 1130. 2411.]
 [  35.  668. 1928.]
 [  26. 1294. 2030.]
 [  26. 1243. 1731.]
 [  13.  752.  823.]
 [   4. 1226. 1690.]
 [   0. 1351. 2243.]
 [   0. 1356. 2159.]
 [   0. 1338. 2159.]
 [   0. 1475. 2531.]]


<!-- Ripley's K Estimators

![RipleyK](/img/ripleyK.png)

![RipleyL](/img/ripleyL.png)

![RipleyH](/img/ripleyH.png) -->

### Run all univariate and bivariate comparisons

In [ ]:
all_points = {
    "tumor": tumor_points,
    "ng2": ng2_points,
    "branch": branch_points,
    "tvc": tvc_points
}

output_dir = "./ripley_results"

# Starting at 2 because radius of 1 causes errors at the boundaries
# where sometimes the entire circle falls outside of study area
# resulting division by zero errors in some instances
radii = np.arange(2, 100) 

# # Run univariate comparisons
# for name, points in all_points.items():
#     print(f"Running univariate analyses on: {name} points")
#     CSR_rstats = monte_carlo(points, mask, radii, n_samples=100, n_processes=55, boundary_correction=False)
#     u_results = run_ripley(points, points, mask, radii, n_processes=55, boundary_correction=False)

#     # Uncomment to save rstats to csv

#     CSR_rstats.to_csv(f"/home/dkermany/ripley_results/{filename}_random_univariate_{name}_rstats.csv")
#     u_rstats.to_csv(os.path.join(output_dir, f"{filename}_univariate_{name}_rstats.csv"))

In [ ]:
# # Run multivariate comparisons with tumor
# for name, points in all_points.items():
#     if name != "tumor":
#         print(f"Running multivariate analyses between tumor and {name} points")
#         rstats = monte_carlo(tumor_points, mask, radii, points, n_samples=100, n_processes=55, boundary_correction=False)
#         m_results = run_ripley(tumor_points, points, mask, radii, n_processes=55, boundary_correction=False)

#         # Uncomment to save rstats to csv
#         CSR_rstats.to_csv(f"/home/dkermany/ripley_results/{filename}_random_multivariate_{name}_rstats.csv")
#         m_rstats.to_csv(os.path.join(output_dir, f"{filename}_multivariate_{name}_rstats.csv"))